# The *t*-distribution

Supplement to the [8.0 Hypothesis testing](8.0-Hypothesis-testing.ipynb) notebook.

* [Preliminaries](#Preliminaries)
* [Evaluating the mean when the uncertainty is estimated from the same data](#Evaluating-the-mean-when-the-uncertainty-is-estimated-from-the-same-data)
    * [Exercise 1](#Exercise-1)
* [Definition of the *t*-distribution](#Definition-of-the-t-distribution)
* [Dependence on the number of degrees of freedom](#Dependence-on-the-number-of-degrees-of-freedom)
* [Corrections to confidence intervals with the t-distribution](#Corrections-to-confidence-intervals-with-the-t-distribution)

## Preliminaries
The following code cell includes the initialization commands needed for this notebook.

In [ ]:
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt
from scipy.stats import t
from scipy.stats import norm

%matplotlib inline

## Evaluating the mean when the uncertainty is estimated from the same data

If we draw $N$ samples, $x_k, k = 0, 1, \ldots, N-1,$ from a normal distribution $\mathcal{N}(x; \mu, \sigma^2)$, then we have seen (*MU* Sec. 2.7) that the arithmetic mean $\bar{x}$ will also be normally distributed, with mean $\mu$ and standard deviation $\sigma/\sqrt{N}$. So far, we have assumed that we know $\sigma$ accurately from an independent set of measurements, but frequently both $\mu$ and $\sigma$ must be estimated from the same data set. This affects how we evaluate the uncertainty in $\bar{x}$, as the next code cell demonstrates.

The figure shows the mean, the true standard error (blue, left), and the estimated standard error (black, right) for `N = 5` normally distributed random numbers over `N_trial = 50` trials. The true standard error is constant across all trials, but the estimated standard error fluctuates with each trial. Most of the time, both intervals either include the true mean, $\mu=0$, or they both exclude it, but in trials $i = 0, 6, 14, 26, 29, 44,$ one interval includes the true mean and the other does not.

In [ ]:
# Set simulation parameters
N_trial = 50
N = 5

# Initialize RNG, choose samples, and assign trial indices
rg = default_rng(0)
X = rg.normal(size=(N, N_trial))
i = np.arange(N_trial)

# Compute mean, estimated standard error, and known standard error
xbar = np.mean(X, axis=0)
alpha_est = np.std(X, axis=0, ddof=1)/np.sqrt(N)
alpha_known = np.ones(N_trial)/np.sqrt(N)

# Create a wide plot
fig, ax = plt.subplots(figsize=[12.8, 4.8])

# Create errorbar plots using known and estimated standard errors,
# shifting each error bar to the left or right by shift_x
shift_x = 0.15
ax.errorbar(i - shift_x, xbar, yerr=alpha_known, fmt='b+')
ax.errorbar(i + shift_x, xbar, yerr=alpha_est, fmt='k+')

# Make a zero line
ax.axhline(c='r')

# Set axis ticks and labels
plt.xticks(np.arange(0, N_trial, 5))
plt.xlabel("Trial")
plt.ylabel(r"$\bar{x}$")

plt.show()

# Test each sample for consistency with the null hypothesis, H0, that
# the mean is consistent with µ = 0 for the given uncertainty
H0_known = (xbar - alpha_known< 0) & (xbar + alpha_known > 0)
H0_est = (xbar - alpha_est < 0) & (xbar + alpha_est > 0)

# Show the fraction of trials consistent with µ = 0 in each case
print("Fraction of trials consistent with µ = 0, known uncertainty:     "
      f"{100*np.sum(H0_known)/N_trial:.1f} %")
print("Fraction of trials consistent with µ = 0, estimated uncertainty: "
      f"{100*np.sum(H0_est)/N_trial:.1f} %")

# Show indices where the known and estimated uncertainties yield different results for H0
print("Trials consistent with µ = 0 for known uncertainty but inconsistent for estimated uncertainty:")
print(np.nonzero(H0_known & ~H0_est)[0])
print("Trials consistent with µ = 0 for estimated uncertainty but inconsistent for known uncertainty:")
print(np.nonzero(~H0_known & H0_est)[0])

### Exercise 1
Run the code cell above for different values of `N` and compare the estimated uncertainties to the true uncertainties. Also, try different values for the seed in `default_rng`. How does `N` influence the average fraction of trials that are consistent with $\mu = 0$?

## Definition of the *t*-distribution

As discussed in *MU* Sec.&nbsp;8.8, the $t$-distribution describes the probability density for the quantity

$$
t = \frac{(\bar{x} - \mu)}{\alpha} = \frac{\sqrt{N}(\bar{x} - \mu)}{\sigma_{N-1}},
$$

which is the *ratio* of two random numbers. The numerator, $\sqrt{N}(\bar{x} - \mu)$, is normally distributed around zero with variance $\sigma^2$. The square of the denominator, $\sigma_{N-1}^2$, is $\chi^2$-distributed with $\nu = N-1$ degrees of freedom.

The next code cell shows the sample distributions for the ratios $z = \sqrt{N}(\bar{x} - \mu)/\sigma$ and $t = \sqrt{N}(\bar{x} - \mu)/\sigma_{N-1}$ for $N = 5$ (which we denote by `N_tsim` to avoid conflict with `N`, above). We choose `N_tsim_trial = 100000` trials to obtain smooth histograms over 50 bins (using the `bins=50` option). The `density=True` option normalizes each bin frequency by the total number of samples, to represent a probability density. We also superpose the PDFs for the standard normal distribution and the $t$-distribution with $\nu = N - 1$ degrees of freedom.

In [ ]:
# Set simulation parameters
N_tsim_trial = 100000
N_tsim = 5

# Initialize RNG and choose samples
rg = default_rng(0)
X_tsim = rg.normal(size=(N_tsim, N_tsim_trial))

# Compute mean, estimated standard error, and known standard error
xbar_tsim = np.mean(X_tsim, axis=0)
alpha_est_tsim = np.std(X_tsim, axis=0, ddof=1)/np.sqrt(N_tsim)
alpha_known_tsim = np.ones(N_tsim_trial)/np.sqrt(N_tsim)

# Show histograms of sample distributions
plt.hist(xbar_tsim/alpha_known_tsim, bins=50, density=True, range=(-5,5), alpha=0.5)
plt.hist(xbar_tsim/alpha_est_tsim, bins=50, density=True, range=(-5,5), alpha=0.5)

# Show PDFs for normal distribution and t-distribution with nu = N_tsim - 1
x = np.linspace(-5, 5, 100)
plt.plot(x, norm.pdf(x))
plt.plot(x, t.pdf(x, N_tsim - 1))

plt.xlabel("x")
plt.ylabel("Frequency")

plt.show()

## Dependence on the number of degrees of freedom
As the histogram above shows, the $t$-distribution is very similar to a normal distribution but has what we call "fatter tails," which decay to zero more gradually as $|x|$ increases. This means that the confidence interval $\bar{x} \pm \sigma_{N-1}/\sqrt{N}$ will exclude the true mean more frequently than we would expect from a random variable drawn from a normal distribution with the same mean and standard error.

The code cell below shows how the shape of the $t$-distribution varies with the number of degrees of freedom $\nu$. The $t$-distribution approaches the standard normal distribution in the limit $\nu\rightarrow\infty$, and the plot shows that the two distributions are already very close for $\nu = 20$, which corresponds to $N = 21$ measurements. This means that you can forget about the $t$-distribution if you know that the measurement uncertainty is fixed, since the average of 21 or more repeated measurements will be described very well by a normal distribution.

And as the plot also shows, even $N = 5$ measurements (with $\nu = N - 1 = 4$) is not bad, and we can compensate for the fat tails by using the $t$-distribution to assign confidence bounds, as discussed in *MU* Sec. 8.8 and in [Corrections to confidence intervals with the $t$-distribution](#Corrections-to-confidence-intervals-with-the-$t$-distribution) below.

In general, $\lim_{|x|\rightarrow\infty} t(x; \nu) \propto x^{-(n + 1)}$ for all finite values of $\nu$. This is known as a "power-law" decay, which approaches zero much more slowly than exponential decay that occurs with the normal distribution—this is the origin of the "fat tails" of the $t$-distribution. The tails are fattest when $\nu = 1$, for which the $t$-distribution is equivalent to another standard distribution known as the [Cauchy distribution](https://en.wikipedia.org/wiki/Cauchy_distribution). The Cauchy distribution has the PDF

$$
P_\text{DF}(x) = \frac{1}{\pi}\frac{1}{1 + x^2},
$$

and it is famously pathological. Although the distribution is symmetric, which suggests that the mean should be zero, the integral that defines the mean is not absolutely convergent; that is,

$$
\int_{-\infty}^{\infty} \left|x P_\text{DF}(x)\right|\,dx \rightarrow \infty.
$$

Consequently, the mean of the Cauchy distribution is not defined. This means that the variance is also undefined, since it describes the mean-squared deviations from the mean. Moreover, the expectation of $x^2$ diverges:

$$
\text{E}(x^2) = \int_{-\infty}^{\infty} x^2 P_\text{DF}(x)\,dx = \frac{1}{\pi}\int_{-\infty}^{\infty} \frac{x^2}{1 + x^2}\,dx \rightarrow \infty
$$

Despite all of these problems, it is still possible to use the $t$-distribution with $\nu=1$ to establish a confidence interval for the average of $N=2$ samples drawn from a normal distribution.

In [ ]:
x = np.linspace(-5, 5, 100)

plt.plot(x, norm.pdf(x))
plt.plot(x, t.pdf(x, 20))
plt.plot(x, t.pdf(x, 4))
plt.plot(x, t.pdf(x, 1))

plt.xlabel("x")
plt.ylabel("P(x)")
plt.legend(["Normal", r"$t(x; \nu = 20)$", r"$t(x; \nu = 4)$", r"$t(x; \nu=1)$ (Cauchy)"])

plt.show()

## Corrections to confidence intervals with the *t*-distribution

In the code cell below we use the `interval` method to reproduce *MU* Fig.&nbsp;8.11. We compute the upper confidence limits for the [`t`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html) distribution at the confidence levels  68&nbsp;%, 90&nbsp;%, and 99&nbsp;%, and compare these to the those obtained from [`norm`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html).

The [`newaxis`](https://numpy.org/doc/stable/reference/constants.html?highlight=newaxis#numpy.newaxis) is a NumPy constant that we use to implement [broadcasting](https://numpy.org/doc/stable/user/theory.broadcasting.html#array-broadcasting-in-numpy), which is a technique that facilitates array computation. The array `alpha` has the shape `(3,)` and the array `nu` has the shape `(28,)`. When we write `t.interval(alpha[:, np.newaxis], nu[np.newaxis, :])`, it tells NumPy to transform each input array into a new array with shape `(3,28)`, in which the elements of `alpha` are replicated along the second dimension and the elements of `nu` are replicated along the first dimension. It then computes `t.interval` elementwise over the entire array.

In [ ]:
# Set confidence levels and degrees of freedom
alpha = np.array([0.68, 0.95, 0.99])
nu = np.concatenate((np.arange(1,20,1), np.arange(20,110,10)))

# Compute upper confidence limits
ci_norm = norm.interval(alpha)[1]
ci_t = t.interval(alpha[:, np.newaxis], nu[np.newaxis, :])[1]

# Plot confidence limits for the t-distribution as markers
plt.semilogx(nu, ci_t[2], '^')
plt.semilogx(nu, ci_t[1], 'o')
plt.semilogx(nu, ci_t[0], 's')

# Show confidence limits for the normal distribution as lines
plt.axhline(y=ci_norm[0], linestyle='--', c='k')
plt.axhline(y=ci_norm[1], linestyle='-.', c='k')
plt.axhline(y=ci_norm[2], linestyle=':', c='k')

# Add legend, labels, and set range for y-axis
plt.legend(["99 %", "90 %", "68 %"])

plt.xlabel(r"Number of degrees of freedom $\nu$")
plt.ylabel("t statistic")
plt.ylim(0, 6)

plt.show()

##### About this notebook
© J. Steven Dodge, 2020. The notebook text is licensed under CC BY 4.0. See more at [Creative Commons](https://creativecommons.org/licenses/by/4.0/). The notebook code is open source under the [MIT License](https://opensource.org/licenses/MIT).